In [1]:
# Museum Visitors vs City Population
#This notebook calls the API to fetch features and model results and renders basic plots.


In [2]:
import requests, pandas as pd, matplotlib.pyplot as plt
BASE = 'http://api:8000'  # Service name in docker-compose
features = requests.get(f'{BASE}/features').json()
features


{'columns': ['museum_id',
  'museum_name',
  'city_id',
  'city_name',
  'year',
  'visitors',
  'population'],
 'rows': [],
 'count': 0}

In [3]:
rows = features.get('rows', [])
cols = features.get('columns', [])
df = pd.DataFrame(rows, columns=cols)
df.head()


,museum_id,museum_name,city_id,city_name,year,visitors,population


In [4]:
model = requests.get(f'{BASE}/model/linear').json()
model


{'model': 'linear_regression',
 'n_samples': 0,
 'r2': None,
 'mae': None,
 'rmse': None,
 'notes': 'No data available. Run ETL first.'}

In [5]:
if not df.empty:
    ax = df.plot.scatter(x='population', y='visitors', title='Population vs Visitors')
    plt.show()
else:
    print('Dataset is empty. Run ETL via the API when available.')


Dataset is empty. Run ETL via the API when available.
